<a href="https://colab.research.google.com/github/jsh1021902/ai_for_machine_learning/blob/main/week5/1_querying_and_merging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Performing Database-style Operations on Dataframes

## About the data
In this notebook, we will using daily weather data that was taken from the [National Centers for Environmental Information (NCEI) API](https://www.ncdc.noaa.gov/cdo-web/webservices/v2). The [`0-weather_data_collection.ipynb`](./0-weather_data_collection.ipynb) notebook contains the process that was followed to collect the data. Consult the dataset's [documentation](https://www1.ncdc.noaa.gov/pub/data/cdo/documentation/GHCND_documentation.pdf) for information on the fields.

*Note: The NCEI is part of the National Oceanic and Atmospheric Administration (NOAA) and, as you can see from the URL for the API, this resource was created when the NCEI was called the NCDC. Should the URL for this resource change in the future, you can search for "NCEI weather API" to find the updated one.*


## Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/Hands-On-Data-Analysis-with-Pandas-2nd-edition/ch_04/

/content/drive/MyDrive/Hands-On-Data-Analysis-with-Pandas-2nd-edition/ch_04


In [3]:
import pandas as pd

weather = pd.read_csv('data/nyc_weather_2018.csv')
weather.head()

,date,datatype,station,attributes,value
0,2018-01-01T00:00:00,PRCP,GHCND:US1CTFR0039,",,N,",0.0
1,2018-01-01T00:00:00,PRCP,GHCND:US1NJBG0015,",,N,",0.0
2,2018-01-01T00:00:00,SNOW,GHCND:US1NJBG0015,",,N,",0.0
3,2018-01-01T00:00:00,PRCP,GHCND:US1NJBG0017,",,N,",0.0
4,2018-01-01T00:00:00,SNOW,GHCND:US1NJBG0017,",,N,",0.0


## Querying DataFrames
The `query()` method is an easier way of filtering based on some criteria. For example, we can use it to find all entries where snow was recorded from a station with `US1NY` in its station ID:

In [4]:
import numpy as np

In [5]:
snow_data = weather.query('datatype == "SNOW" and value > 0 and station.str.contains("US1NY")')
snow_data.head()

,date,datatype,station,attributes,value
114,2018-01-01T00:00:00,SNOW,GHCND:US1NYWC0019,",,N,",25.0
789,2018-01-04T00:00:00,SNOW,GHCND:US1NYNS0007,",,N,",41.0
794,2018-01-04T00:00:00,SNOW,GHCND:US1NYNS0018,",,N,",10.0
798,2018-01-04T00:00:00,SNOW,GHCND:US1NYNS0024,",,N,",89.0
800,2018-01-04T00:00:00,SNOW,GHCND:US1NYNS0030,",,N,",102.0


This is equivalent to querying the `weather.db` SQLite database for

```sql
SELECT *
FROM weather
WHERE datatype == "SNOW" AND value > 0 AND station LIKE "%US1NY%"
```

In [6]:
import sqlite3

with sqlite3.connect('data/weather.db') as connection:
    snow_data_from_db = pd.read_sql(
        'SELECT * FROM weather WHERE datatype == "SNOW" AND value > 0 and station LIKE "%US1NY%"',
        connection
    )

snow_data.reset_index().drop(columns='index').equals(snow_data_from_db)

True

Note this is also equivalent to creating Boolean masks:

In [7]:
weather[
    (weather.datatype == 'SNOW')
    & (weather.value > 0)
    & weather.station.str.contains('US1NY')
].equals(snow_data)

True

In [8]:
weather[
    (weather.datatype == 'SNOW')
    & (weather.value > 0)
    & weather.station.str.contains('US1NY')
]

,date,datatype,station,attributes,value
114,2018-01-01T00:00:00,SNOW,GHCND:US1NYWC0019,",,N,",25.0
789,2018-01-04T00:00:00,SNOW,GHCND:US1NYNS0007,",,N,",41.0
794,2018-01-04T00:00:00,SNOW,GHCND:US1NYNS0018,",,N,",10.0
798,2018-01-04T00:00:00,SNOW,GHCND:US1NYNS0024,",,N,",89.0
800,2018-01-04T00:00:00,SNOW,GHCND:US1NYNS0030,",,N,",102.0
...,...,...,...,...,...
68631,2018-11-16T00:00:00,SNOW,GHCND:US1NYRL0005,",,N,",170.0
68637,2018-11-16T00:00:00,SNOW,GHCND:US1NYWC0018,",,N,",191.0
74879,2018-12-14T00:00:00,SNOW,GHCND:US1NYWC0018,",,N,",3.0
77120,2018-12-24T00:00:00,SNOW,GHCND:US1NYWC0018,",,N,",18.0


In [9]:
weather.loc[
    (weather.datatype == 'SNOW')
    & (weather.value > 0)
    & weather.station.str.contains('US1NY') , :
]

,date,datatype,station,attributes,value
114,2018-01-01T00:00:00,SNOW,GHCND:US1NYWC0019,",,N,",25.0
789,2018-01-04T00:00:00,SNOW,GHCND:US1NYNS0007,",,N,",41.0
794,2018-01-04T00:00:00,SNOW,GHCND:US1NYNS0018,",,N,",10.0
798,2018-01-04T00:00:00,SNOW,GHCND:US1NYNS0024,",,N,",89.0
800,2018-01-04T00:00:00,SNOW,GHCND:US1NYNS0030,",,N,",102.0
...,...,...,...,...,...
68631,2018-11-16T00:00:00,SNOW,GHCND:US1NYRL0005,",,N,",170.0
68637,2018-11-16T00:00:00,SNOW,GHCND:US1NYWC0018,",,N,",191.0
74879,2018-12-14T00:00:00,SNOW,GHCND:US1NYWC0018,",,N,",3.0
77120,2018-12-24T00:00:00,SNOW,GHCND:US1NYWC0018,",,N,",18.0


## Merging DataFrames
We have data for many different stations each day; however, we don't know what the stations are&mdash;just their IDs. We can join the data in the `weather_stations.csv` file which contains information from the `stations` endpoint of the NCEI API. Consult the [`0-weather_data_collection.ipynb`](./0-weather_data_collection.ipynb) notebook to see how this was collected. It looks like this:

In [10]:
station_info = pd.read_csv('data/weather_stations.csv')
station_info.head()

,id,name,latitude,longitude,elevation
0,GHCND:US1CTFR0022,"STAMFORD 2.6 SSW, CT US",41.064100,-73.577000,36.6
1,GHCND:US1CTFR0039,"STAMFORD 4.2 S, CT US",41.037788,-73.568176,6.4
2,GHCND:US1NJBG0001,"BERGENFIELD 0.3 SW, NJ US",40.921298,-74.001983,20.1
3,GHCND:US1NJBG0002,"SADDLE BROOK TWP 0.6 E, NJ US",40.902694,-74.083358,16.8
4,GHCND:US1NJBG0003,"TENAFLY 1.3 W, NJ US",40.914670,-73.977500,21.6


As a reminder, the weather data looks like this:

In [11]:
weather.head()

,date,datatype,station,attributes,value
0,2018-01-01T00:00:00,PRCP,GHCND:US1CTFR0039,",,N,",0.0
1,2018-01-01T00:00:00,PRCP,GHCND:US1NJBG0015,",,N,",0.0
2,2018-01-01T00:00:00,SNOW,GHCND:US1NJBG0015,",,N,",0.0
3,2018-01-01T00:00:00,PRCP,GHCND:US1NJBG0017,",,N,",0.0
4,2018-01-01T00:00:00,SNOW,GHCND:US1NJBG0017,",,N,",0.0


We can join our data by matching up the `station_info.id` column with the `weather.station` column. Before doing that though, let's see how many unique values we have:

In [12]:
station_info.id.describe()

count                   279
unique                  279
top       GHCND:US1CTFR0022
freq                      1
Name: id, dtype: object

While `station_info` has one row per station, the `weather` dataframe has many entries per station. Notice it also has fewer uniques:

In [13]:
weather.station.describe()

count                 78780
unique                  110
top       GHCND:USW00094789
freq                   4270
Name: station, dtype: object

When working with joins, it is important to keep an eye on the row count. Some join types will lead to data loss. Remember that we can get this with `shape`:

In [14]:
station_info.shape[0], weather.shape[0]

(279, 78780)

Since we will be doing this often, it makes more sense to write a function:

In [15]:
def get_row_count(*dfs):
    return [df.shape[0] for df in dfs]
get_row_count(station_info, weather)

[279, 78780]

By default, `merge()` performs an inner join. We simply specify the columns to use for the join. The left dataframe is the one we call `merge()` on, and the right one is passed in as an argument:

In [16]:
inner_join = weather.merge(station_info, left_on='station', right_on='id')
inner_join.sample(5, random_state=0)

,date,datatype,station,attributes,value,id,name,latitude,longitude,elevation
10739,2018-08-07T00:00:00,SNOW,GHCND:US1NJMN0069,",,N,",0.0,GHCND:US1NJMN0069,"LONG BRANCH 1.7 SSW, NJ US",40.275368,-74.006027,9.4
45188,2018-12-21T00:00:00,TMAX,GHCND:USW00014732,",,W,2400",16.7,GHCND:USW00014732,"LAGUARDIA AIRPORT, NY US",40.779440,-73.880350,3.4
59823,2018-01-15T00:00:00,WDF5,GHCND:USW00094741,",,W,",40.0,GHCND:USW00094741,"TETERBORO AIRPORT, NJ US",40.850000,-74.061390,2.7
10852,2018-10-31T00:00:00,PRCP,GHCND:US1NJMN0069,"T,,N,",0.0,GHCND:US1NJMN0069,"LONG BRANCH 1.7 SSW, NJ US",40.275368,-74.006027,9.4
46755,2018-05-05T00:00:00,SNOW,GHCND:USW00014734,",,W,",0.0,GHCND:USW00014734,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",40.682500,-74.169400,2.1


We can remove the duplication of information in the `station` and `id` columns by renaming one of them before the merge and then simply using `on`:

In [18]:
station_info.rename(dict(id='station'), axis=1)

,station,name,latitude,longitude,elevation
0,GHCND:US1CTFR0022,"STAMFORD 2.6 SSW, CT US",41.064100,-73.577000,36.6
1,GHCND:US1CTFR0039,"STAMFORD 4.2 S, CT US",41.037788,-73.568176,6.4
2,GHCND:US1NJBG0001,"BERGENFIELD 0.3 SW, NJ US",40.921298,-74.001983,20.1
3,GHCND:US1NJBG0002,"SADDLE BROOK TWP 0.6 E, NJ US",40.902694,-74.083358,16.8
4,GHCND:US1NJBG0003,"TENAFLY 1.3 W, NJ US",40.914670,-73.977500,21.6
...,...,...,...,...,...
274,GHCND:USW00054787,"FARMINGDALE REPUBLIC AIRPORT, NY US",40.734170,-73.416940,24.7
275,GHCND:USW00094728,"NY CITY CENTRAL PARK, NY US",40.778980,-73.969250,42.7
276,GHCND:USW00094741,"TETERBORO AIRPORT, NJ US",40.850000,-74.061390,2.7
277,GHCND:USW00094745,"WESTCHESTER CO AIRPORT, NY US",41.062360,-73.704630,111.9


In [17]:
weather.merge(station_info.rename(dict(id='station'), axis=1), on='station').sample(5, random_state=0)

,date,datatype,station,attributes,value,name,latitude,longitude,elevation
10739,2018-08-07T00:00:00,SNOW,GHCND:US1NJMN0069,",,N,",0.0,"LONG BRANCH 1.7 SSW, NJ US",40.275368,-74.006027,9.4
45188,2018-12-21T00:00:00,TMAX,GHCND:USW00014732,",,W,2400",16.7,"LAGUARDIA AIRPORT, NY US",40.779440,-73.880350,3.4
59823,2018-01-15T00:00:00,WDF5,GHCND:USW00094741,",,W,",40.0,"TETERBORO AIRPORT, NJ US",40.850000,-74.061390,2.7
10852,2018-10-31T00:00:00,PRCP,GHCND:US1NJMN0069,"T,,N,",0.0,"LONG BRANCH 1.7 SSW, NJ US",40.275368,-74.006027,9.4
46755,2018-05-05T00:00:00,SNOW,GHCND:USW00014734,",,W,",0.0,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US",40.682500,-74.169400,2.1


We are losing stations that don't have weather observations associated with them, if we don't want to lose these rows, we perform a right or left join instead of the inner join:

In [19]:
left_join = station_info.merge(weather, left_on='id', right_on='station', how='left')
right_join = weather.merge(station_info, left_on='station', right_on='id', how='right')

right_join[right_join.datatype.isna()].head()

,date,datatype,station,attributes,value,id,name,latitude,longitude,elevation
0,NaN,NaN,NaN,NaN,NaN,GHCND:US1CTFR0022,"STAMFORD 2.6 SSW, CT US",41.064100,-73.577000,36.6
344,NaN,NaN,NaN,NaN,NaN,GHCND:US1NJBG0001,"BERGENFIELD 0.3 SW, NJ US",40.921298,-74.001983,20.1
345,NaN,NaN,NaN,NaN,NaN,GHCND:US1NJBG0002,"SADDLE BROOK TWP 0.6 E, NJ US",40.902694,-74.083358,16.8
718,NaN,NaN,NaN,NaN,NaN,GHCND:US1NJBG0005,"WESTWOOD 0.8 ESE, NJ US",40.983041,-74.015858,15.8
719,NaN,NaN,NaN,NaN,NaN,GHCND:US1NJBG0006,"RAMSEY 0.6 E, NJ US",41.058611,-74.134068,112.2


In [20]:
right_join.head()

,date,datatype,station,attributes,value,id,name,latitude,longitude,elevation
0,NaN,NaN,NaN,NaN,NaN,GHCND:US1CTFR0022,"STAMFORD 2.6 SSW, CT US",41.064100,-73.577000,36.6
1,2018-01-01T00:00:00,PRCP,GHCND:US1CTFR0039,",,N,",0.0,GHCND:US1CTFR0039,"STAMFORD 4.2 S, CT US",41.037788,-73.568176,6.4
2,2018-01-02T00:00:00,PRCP,GHCND:US1CTFR0039,",,N,",0.0,GHCND:US1CTFR0039,"STAMFORD 4.2 S, CT US",41.037788,-73.568176,6.4
3,2018-01-03T00:00:00,PRCP,GHCND:US1CTFR0039,",,N,",0.0,GHCND:US1CTFR0039,"STAMFORD 4.2 S, CT US",41.037788,-73.568176,6.4
4,2018-01-05T00:00:00,DAPR,GHCND:US1CTFR0039,",,N,",2.0,GHCND:US1CTFR0039,"STAMFORD 4.2 S, CT US",41.037788,-73.568176,6.4


The left and right join as we performed above are equivalent because the side for which we kept the rows without matches was the same in both cases:

In [21]:
left_join.sort_index(axis=1).sort_values(['date', 'station'], ignore_index=True).equals(
    right_join.sort_index(axis=1).sort_values(['date', 'station'], ignore_index=True)
)

True

Note we have additional rows in the left and right joins because we kept all the stations that didn't have weather observations:

In [22]:
get_row_count(inner_join, left_join, right_join)

[78780, 78949, 78949]

If we query the station information for stations that have `US1NY` in their ID and perform an outer join, we can see where the mismatches occur:

In [23]:
outer_join = weather.merge(
    station_info[station_info.id.str.contains('US1NY')],
    left_on='station', right_on='id', how='outer', indicator=True
)

pd.concat([
    outer_join.query(f'_merge == "{kind}"').sample(2, random_state=0)
    for kind in outer_join._merge.unique()
]).sort_index()

,date,datatype,station,attributes,value,id,name,latitude,longitude,elevation,_merge
23634,2018-04-12T00:00:00,PRCP,GHCND:US1NYNS0043,",,N,",0.0,GHCND:US1NYNS0043,"PLAINVIEW 0.4 ENE, NY US",40.785919,-73.466873,56.7,both
25742,2018-03-25T00:00:00,PRCP,GHCND:US1NYSF0061,",,N,",0.0,GHCND:US1NYSF0061,"CENTERPORT 0.9 SW, NY US",40.891689,-73.383133,53.6,both
60645,2018-04-16T00:00:00,TMIN,GHCND:USW00094741,",,W,",3.9,NaN,NaN,NaN,NaN,NaN,left_only
70764,2018-03-23T00:00:00,SNWD,GHCND:US1NJHD0002,",,N,",203.0,NaN,NaN,NaN,NaN,NaN,left_only
78790,NaN,NaN,NaN,NaN,NaN,GHCND:US1NYQN0033,"HOWARD BEACH 0.4 NNW, NY US",40.662099,-73.841345,2.1,right_only
78800,NaN,NaN,NaN,NaN,NaN,GHCND:US1NYWC0009,"NEW ROCHELLE 1.3 S, NY US",40.904000,-73.777000,21.9,right_only


These joins are equivalent to their SQL counterparts. Below is the inner join. Note that to use `equals()` you will have to do some manipulation of the dataframes to line them up:

In [24]:
import sqlite3

with sqlite3.connect('data/weather.db') as connection:
    inner_join_from_db = pd.read_sql(
        'SELECT * FROM weather JOIN stations ON weather.station == stations.id',
        connection
    )

inner_join_from_db.shape == inner_join.shape

True

Revisiting the dirty data from chapter 3's [`5-handling_data_issues.ipynb`](../ch_03/5-handling_data_issues.ipynb) notebook.

Data meanings:
- `PRCP`: precipitation in millimeters
- `SNOW`: snowfall in millimeters
- `SNWD`: snow depth in millimeters
- `TMAX`: maximum daily temperature in Celsius
- `TMIN`: minimum daily temperature in Celsius
- `TOBS`: temperature at time of observation in Celsius
- `WESF`: water equivalent of snow in millimeters


Read in the data, dropping duplicates and the uninformative `SNWD` column:

In [25]:
dirty_data = pd.read_csv(
    'data/dirty_data.csv', index_col='date'
).drop_duplicates().drop(columns='SNWD')
dirty_data.head()

,station,PRCP,SNOW,TMAX,TMIN,TOBS,WESF,inclement_weather
date,,,,,,,,
2018-01-01T00:00:00,?,0.0,0.0,5505.0,-40.0,NaN,NaN,NaN
2018-01-02T00:00:00,GHCND:USC00280907,0.0,0.0,-8.3,-16.1,-12.2,NaN,False
2018-01-03T00:00:00,GHCND:USC00280907,0.0,0.0,-4.4,-13.9,-13.3,NaN,False
2018-01-04T00:00:00,?,20.6,229.0,5505.0,-40.0,NaN,19.3,True
2018-01-05T00:00:00,?,0.3,NaN,5505.0,-40.0,NaN,NaN,NaN


We need to create two dataframes for the join. We will drop some unecessary columns as well for easier viewing:

In [26]:
valid_station = dirty_data.query('station != "?"').drop(columns=['WESF', 'station'])
station_with_wesf = dirty_data.query('station == "?"').drop(columns=['station', 'TOBS', 'TMIN', 'TMAX'])

Our column for the join is the index in both dataframes, so we must specify `left_index` and `right_index`:

In [27]:
valid_station.merge(
    station_with_wesf, how='left', left_index=True, right_index=True
).query('WESF > 0').head()

,PRCP_x,SNOW_x,TMAX,TMIN,TOBS,inclement_weather_x,PRCP_y,SNOW_y,WESF,inclement_weather_y
date,,,,,,,,,,
2018-01-30T00:00:00,0.0,0.0,6.7,-1.7,-0.6,False,1.5,13.0,1.8,True
2018-03-08T00:00:00,48.8,NaN,1.1,-0.6,1.1,False,28.4,NaN,28.7,NaN
2018-03-13T00:00:00,4.1,51.0,5.6,-3.9,0.0,True,3.0,13.0,3.0,True
2018-03-21T00:00:00,0.0,0.0,2.8,-2.8,0.6,False,6.6,114.0,8.6,True
2018-04-02T00:00:00,9.1,127.0,12.8,-1.1,-1.1,True,14.0,152.0,15.2,True


The columns that existed in both dataframes, but didn't form part of the join got suffixes added to their names: `_x` for columns from the left dataframe and `_y` for columns from the right dataframe. We can customize this with the `suffixes` argument:

In [28]:
valid_station.merge(
    station_with_wesf, how='left', left_index=True, right_index=True, suffixes=('', '_?')
).query('WESF > 0').head()

,PRCP,SNOW,TMAX,TMIN,TOBS,inclement_weather,PRCP_?,SNOW_?,WESF,inclement_weather_?
date,,,,,,,,,,
2018-01-30T00:00:00,0.0,0.0,6.7,-1.7,-0.6,False,1.5,13.0,1.8,True
2018-03-08T00:00:00,48.8,NaN,1.1,-0.6,1.1,False,28.4,NaN,28.7,NaN
2018-03-13T00:00:00,4.1,51.0,5.6,-3.9,0.0,True,3.0,13.0,3.0,True
2018-03-21T00:00:00,0.0,0.0,2.8,-2.8,0.6,False,6.6,114.0,8.6,True
2018-04-02T00:00:00,9.1,127.0,12.8,-1.1,-1.1,True,14.0,152.0,15.2,True


Since we are joining on the index, an easier way is to use the `join()` method instead of `merge()`. Note that the suffix parameter is now `lsuffix` for the left dataframe's suffix and `rsuffix` for the right one's:

In [29]:
valid_station.join(station_with_wesf, how='left', rsuffix='_?').query('WESF > 0').head()

,PRCP,SNOW,TMAX,TMIN,TOBS,inclement_weather,PRCP_?,SNOW_?,WESF,inclement_weather_?
date,,,,,,,,,,
2018-01-30T00:00:00,0.0,0.0,6.7,-1.7,-0.6,False,1.5,13.0,1.8,True
2018-03-08T00:00:00,48.8,NaN,1.1,-0.6,1.1,False,28.4,NaN,28.7,NaN
2018-03-13T00:00:00,4.1,51.0,5.6,-3.9,0.0,True,3.0,13.0,3.0,True
2018-03-21T00:00:00,0.0,0.0,2.8,-2.8,0.6,False,6.6,114.0,8.6,True
2018-04-02T00:00:00,9.1,127.0,12.8,-1.1,-1.1,True,14.0,152.0,15.2,True


Joins can be very resource-intensive, so it's a good idea to figure out what type of join you need using set operations before trying the join itself. The `pandas` set operations are performed on the index, so whichever columns we will be joining on will need to be the index. Let's go back to the `weather` and `station_info` dataframes and set the station ID columns as the index:

In [30]:
weather.set_index('station', inplace=True)
station_info.set_index('id', inplace=True)

The intersection will tell us the stations that are present in both dataframes. The result will be the index when performing an inner join:

In [31]:
weather.index.intersection(station_info.index)

Index(['GHCND:US1CTFR0039', 'GHCND:US1NJBG0015', 'GHCND:US1NJBG0017',
       'GHCND:US1NJBG0018', 'GHCND:US1NJBG0023', 'GHCND:US1NJBG0030',
       'GHCND:US1NJBG0039', 'GHCND:US1NJBG0044', 'GHCND:US1NJES0018',
       'GHCND:US1NJES0024',
       ...
       'GHCND:US1NJBG0037', 'GHCND:USC00284987', 'GHCND:US1NJES0031',
       'GHCND:US1NJES0029', 'GHCND:US1NJMD0086', 'GHCND:US1NJMS0097',
       'GHCND:US1NJMN0081', 'GHCND:US1NJMD0088', 'GHCND:US1NJES0040',
       'GHCND:US1NYQN0029'],
      dtype='object', length=110)

The set difference will tell us what we lose from each side. When performing an inner join, we lose nothing from the `weather` dataframe:

In [32]:
weather.index.difference(station_info.index)

Index([], dtype='object')

We lose 169 stations from the `station_info` dataframe, however:

In [33]:
station_info.index.difference(weather.index)

Index(['GHCND:US1CTFR0022', 'GHCND:US1NJBG0001', 'GHCND:US1NJBG0002',
       'GHCND:US1NJBG0005', 'GHCND:US1NJBG0006', 'GHCND:US1NJBG0008',
       'GHCND:US1NJBG0011', 'GHCND:US1NJBG0012', 'GHCND:US1NJBG0013',
       'GHCND:US1NJBG0020',
       ...
       'GHCND:USC00308322', 'GHCND:USC00308749', 'GHCND:USC00308946',
       'GHCND:USC00309117', 'GHCND:USC00309270', 'GHCND:USC00309400',
       'GHCND:USC00309466', 'GHCND:USC00309576', 'GHCND:USW00014708',
       'GHCND:USW00014786'],
      dtype='object', length=169)

The symmetric difference tells us what we lose from both sides. It is the combination of the set differences in each direction:

In [34]:
ny_in_name = station_info[station_info.index.str.contains('US1NY')]

ny_in_name.index.difference(weather.index).shape[0]\
+ weather.index.difference(ny_in_name.index).shape[0]\
== weather.index.symmetric_difference(ny_in_name.index).shape[0]

True

The union will show us everything that will be present after a full outer join. Note that we pass in the unique values of the index to make sure we can see the number of stations we will be left with:

In [35]:
weather.index.unique().union(station_info.index)

Index(['GHCND:US1CTFR0022', 'GHCND:US1CTFR0039', 'GHCND:US1NJBG0001',
       'GHCND:US1NJBG0002', 'GHCND:US1NJBG0003', 'GHCND:US1NJBG0005',
       'GHCND:US1NJBG0006', 'GHCND:US1NJBG0008', 'GHCND:US1NJBG0010',
       'GHCND:US1NJBG0011',
       ...
       'GHCND:USW00014708', 'GHCND:USW00014732', 'GHCND:USW00014734',
       'GHCND:USW00014786', 'GHCND:USW00054743', 'GHCND:USW00054787',
       'GHCND:USW00094728', 'GHCND:USW00094741', 'GHCND:USW00094745',
       'GHCND:USW00094789'],
      dtype='object', length=279)

Note that the symmetric difference is actually the union of the set differences:

In [36]:
ny_in_name = station_info[station_info.index.str.contains('US1NY')]

ny_in_name.index.difference(weather.index).union(weather.index.difference(ny_in_name.index)).equals(
    weather.index.symmetric_difference(ny_in_name.index)
)

True

<hr>
<div>
    <a href="../ch_03/5-handling_data_issues.ipynb">
        <button>&#8592; Chapter 3</button>
    </a>
    <a href="./0-weather_data_collection.ipynb">
        <button>Weather Data Collection</button>
    </a>
    <a href="./2-dataframe_operations.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<hr>